Для начала загрузим данные ЕГЭ, на которых проверим модель:

In [1]:
from ast import literal_eval

import pandas as pd

from google import genai
from openai import OpenAI
from sklearn.metrics import f1_score, accuracy_score, classification_report

In [ ]:
df = pd.read_excel("../data_input/EgeEvalDataset.xlsx")
df.head()

,index,reading_text,question,right_answer,distractors
0,0,На задворках нашего села стояло на сваях длин...,Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?,Полонез вызвал у автора желание заплакать и с...,"[' Рассказчик был сиротой.', ' В детстве эта м..."
1,1,"Мы ехали берегом Лены на юг, а зима догоняла ...",Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?,"Подобравшись ближе к берегу, козы бросились к...","[' Собеседник рассказчика, Сокольский, сомнева..."
2,2,"Воспалённое состояние Поли, а главное, её сби...",Какое высказывание СООТВЕТСТВУЕТ тексту?,Автор письма хранит подарок девочки.,[' Родион встретил девочку перед наступлением....
3,3,"Все мы любили «классного», хотя нельзя сказат...",Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?,Ребята сразу невзлюбили своего классного руко...,[' Белый билет не давал учителю возможности уй...
4,4,В суровые военные годы во время бомбёжки моя ...,Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?,Бабушка рассказчицы долгое время работала в т...,"[' Убежище, в котором укрывалась рассказчица, ..."


In [ ]:
API_KEY = "ENTER-YOUR-API-KEY"

In [4]:
item = df.iloc[0]

In [5]:
literal_eval(item["distractors"])

[' Рассказчик был сиротой.',
 ' В детстве эта мелодия вызывала другие чувства.',
 ' Это музыкальное произведение было написано на прощание с Родиной.']

In [ ]:
client = OpenAI(api_key=API_KEY, base_url="ENTER-YOUR-PROVIDER-URL")

def attest_item_chatgpt4(item: dict) -> list[str]:
    results = []
    distractors = [s.strip() for s in literal_eval(item["distractors"])]
    for option in [item["right_answer"], *distractors]:
        response = client.responses.create(
            model="gpt-4o",
            instructions="Ты - ассистент, который анализирует текстовые отрывки и отвечает на вопросы по их содержанию односложно (Да или Нет)",
            input=f"{item['reading_text']} ВОПРОС: ВЕРНО ЛИ СЛЕДУЮЩЕЕ УТВЕРЖДЕНИЕ? {option}",
            temperature=0.0
        )
        result = response.output_text
        results.append(result)
    return results

In [7]:
attest_item_chatgpt4(item)

['Да.', 'Нет.', 'Да.', 'Да.']

In [ ]:
TRUE_LABELS = []

for idx, row in df.iterrows():
    if row["question"] == "Какое высказывание СООТВЕТСТВУЕТ тексту?":
        TRUE_LABELS.append(1)
        for d in literal_eval(row["distractors"]):
            TRUE_LABELS.append(0)
    elif row["question"] == "Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?":
        TRUE_LABELS.append(0)
        for d in literal_eval(row["distractors"]):
            TRUE_LABELS.append(1)
    else:
        raise ValueError(row["question"])

4 [0, 1, 1, 1]
8 [0, 1, 1, 1, 0, 1, 1, 1]
12 [0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0]
16 [0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1]
20 [0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1]
24 [0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1]
28 [0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0]
32 [0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1]
36 [0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1]
41 [0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1]
45 [0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0]
49 [0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1]
53 [0, 1,

In [9]:
TRUE_LABELS[8:12]

[1, 0, 0, 0]

In [10]:
pred_labels_chatgpt4o = []

for idx, row in df.iterrows():
    answers = attest_item_chatgpt4(row)
    for answer in answers:
        if answer.lower().strip().startswith("да"):
            pred_labels_chatgpt4o.append(1)
        elif answer.lower().strip().startswith("нет"):
            pred_labels_chatgpt4o.append(0)
        else:
            raise ValueError(answer)

In [11]:
f1_score(TRUE_LABELS, pred_labels_chatgpt4o)

0.9389312977099237

In [12]:
accuracy_score(TRUE_LABELS, pred_labels_chatgpt4o)

0.9276018099547512

In [13]:
classification_report(TRUE_LABELS, pred_labels_chatgpt4o, output_dict=True)

{'0': {'precision': 0.9647058823529412,
  'recall': 0.8631578947368421,
  'f1-score': 0.9111111111111111,
  'support': 95.0},
 '1': {'precision': 0.9044117647058824,
  'recall': 0.9761904761904762,
  'f1-score': 0.9389312977099237,
  'support': 126.0},
 'accuracy': 0.9276018099547512,
 'macro avg': {'precision': 0.9345588235294118,
  'recall': 0.9196741854636592,
  'f1-score': 0.9250212044105174,
  'support': 221.0},
 'weighted avg': {'precision': 0.9303300505722651,
  'recall': 0.9276018099547512,
  'f1-score': 0.9269723939683527,
  'support': 221.0}}

In [ ]:
client = OpenAI(
    api_key=API_KEY,
    base_url="ENTER-YOUR-PROVIDER-URL",
)

def attest_item_deepseek(item: dict) -> list[str]:
    results = []
    distractors = [s.strip() for s in literal_eval(item["distractors"])]
    for option in [item["right_answer"], *distractors]:
        messages = [
            {"role": "system", "content": "Ты - ассистент, который анализирует текстовые отрывки и односложно (Да или Нет) отвечает на вопросы по их содержанию"},
            {"role": "user", "content": f"{item['reading_text']} ВОПРОС: ВЕРНО ЛИ СЛЕДУЮЩЕЕ УТВЕРЖДЕНИЕ? {option}"}
        ]
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=messages,
            stream=False,
            temperature=0.0,
            seed=42
        )
        result = response.choices[0].message.content
        results.append(result)
    return results

In [15]:
pred_labels_deepseek = []

for idx, row in df.iterrows():
    answers = attest_item_deepseek(row)
    for answer in answers:
        if answer.lower().strip().startswith("да"):
            pred_labels_deepseek.append(1)
        elif answer.lower().strip().startswith("нет"):
            pred_labels_deepseek.append(0)
        else:
            raise ValueError(answer)

In [16]:
classification_report(TRUE_LABELS, pred_labels_deepseek, output_dict=True)

{'0': {'precision': 0.9879518072289156,
  'recall': 0.8631578947368421,
  'f1-score': 0.9213483146067416,
  'support': 95.0},
 '1': {'precision': 0.9057971014492754,
  'recall': 0.9920634920634921,
  'f1-score': 0.946969696969697,
  'support': 126.0},
 'accuracy': 0.9366515837104072,
 'macro avg': {'precision': 0.9468744543390955,
  'recall': 0.9276106934001671,
  'f1-score': 0.9341590057882193,
  'support': 221.0},
 'weighted avg': {'precision': 0.9411124727120166,
  'recall': 0.9366515837104072,
  'f1-score': 0.9359559805693316,
  'support': 221.0}}

In [ ]:
client = genai.Client(
    api_key=API_KEY, http_options={"base_url": "ENTER-YOUR-PROVIDER-URL"}
)

In [18]:
response = client.models.generate_content(
    model="gemini-2.5-pro-preview-05-06",
    contents="Кто такой Платон Аркадьевич Лесовозов?"
)

In [19]:
print(response.text)

**Платон Аркадьевич Лесовозов** – это вымышленный персонаж, герой детективного романа братьев Вайнеров **"Эра милосердия"** и его знаменитой экранизации, пятисерийного телефильма **"Место встречи изменить нельзя"** (1979 г., режиссер Станислав Говорухин).

В фильме его роль исполнил актер **Всеволод Сафонов**.

**Ключевые характеристики персонажа:**

*   **Профессия:** Прокурор, позднее – следователь по особо важным делам Прокуратуры СССР.
*   **Роль в сюжете:** Лесовозов появляется в ключевой момент, когда расследуется дело об убийстве Ларисы Груздевой. Он представляет собой воплощение закона, справедливости и методичного подхода к расследованию, в отличие от более жестких и порой спорных методов Глеба Жеглова.
*   **Характер:** Умный, принципиальный, спокойный, рассудительный. Он не идет на компромиссы с совестью и требует неукоснительного соблюдения законности.

Платон Аркадьевич Лесовозов – важный второстепенный персонаж, который помогает раскрыть характеры главных героев (Жеглова 

In [20]:
def attest_item_gemini(item: dict) -> list[str]:
    results = []
    distractors = [s.strip() for s in literal_eval(item["distractors"])]
    for option in [item["right_answer"], *distractors]:
        response = client.models.generate_content(
            model="gemini-2.5-pro-preview-05-06",
            contents=f"Прочитай текст и ответь на вопрос односложно (Да или Нет). ТЕКСТ: {item['reading_text']} ВОПРОС: ВЕРНО ЛИ СЛЕДУЮЩЕЕ УТВЕРЖДЕНИЕ? {option}"
        )
        result = response.text
        results.append(result)
    return results

In [21]:
pred_labels_gemini = []

for idx, row in df.iterrows():
    answers = attest_item_gemini(row)
    for answer in answers:
        if answer.lower().strip().startswith("да"):
            pred_labels_gemini.append(1)
        elif answer.lower().strip().startswith("нет"):
            pred_labels_gemini.append(0)
        else:
            raise ValueError(answer)

In [22]:
classification_report(TRUE_LABELS, pred_labels_gemini, output_dict=True)

{'0': {'precision': 0.9381443298969072,
  'recall': 0.9578947368421052,
  'f1-score': 0.9479166666666666,
  'support': 95.0},
 '1': {'precision': 0.967741935483871,
  'recall': 0.9523809523809523,
  'f1-score': 0.96,
  'support': 126.0},
 'accuracy': 0.9547511312217195,
 'macro avg': {'precision': 0.9529431326903891,
  'recall': 0.9551378446115288,
  'f1-score': 0.9539583333333332,
  'support': 221.0},
 'weighted avg': {'precision': 0.9550189828559905,
  'recall': 0.9547511312217195,
  'f1-score': 0.9548058069381599,
  'support': 221.0}}

In [28]:
print(len(TRUE_LABELS))

221


In [29]:
ans_cnt = 0

for idx, row in df.iterrows():
    distractors = [s.strip() for s in literal_eval(row["distractors"])]
    options = [row["right_answer"], *distractors]

    len_options = len(options)

    print(ans_cnt, ans_cnt+len_options, TRUE_LABELS[ans_cnt:ans_cnt+len_options])

    df.loc[idx, "true_labels"] = str(TRUE_LABELS[ans_cnt:ans_cnt+len_options])
    df.loc[idx, "chatgpt_labels"] = str(pred_labels_chatgpt4o[
        ans_cnt:ans_cnt+len_options
    ])
    df.loc[idx, "deepseek_labels"] = str(pred_labels_deepseek[
        ans_cnt:ans_cnt+len_options
    ])
    df.loc[idx, "gemini_labels"] = str(pred_labels_gemini[
        ans_cnt:ans_cnt+len_options
    ])
    ans_cnt += len_options

0 4 [0, 1, 1, 1]
4 8 [0, 1, 1, 1]
8 12 [1, 0, 0, 0]
12 16 [0, 1, 1, 1]
16 20 [0, 1, 1, 1]
20 24 [0, 1, 1, 1]
24 28 [1, 0, 0, 0]
28 32 [0, 1, 1, 1]
32 36 [0, 1, 1, 1]
36 41 [0, 1, 1, 1, 1]
41 45 [1, 0, 0, 0]
45 49 [0, 1, 1, 1]
49 53 [1, 0, 0, 0]
53 57 [0, 1, 1, 1]
57 61 [0, 1, 1, 1]
61 65 [0, 1, 1, 1]
65 69 [1, 0, 0, 0]
69 73 [0, 1, 1, 1]
73 77 [0, 1, 1, 1]
77 81 [0, 1, 1, 1]
81 85 [1, 0, 0, 0]
85 89 [0, 1, 1, 1]
89 93 [0, 1, 1, 1]
93 97 [0, 1, 1, 1]
97 101 [0, 1, 1, 1]
101 105 [1, 0, 0, 0]
105 109 [0, 1, 1, 1]
109 113 [0, 1, 1, 1]
113 117 [1, 0, 0, 0]
117 121 [1, 0, 0, 0]
121 125 [1, 0, 0, 0]
125 129 [1, 0, 0, 0]
129 133 [0, 1, 1, 1]
133 137 [1, 0, 0, 0]
137 141 [0, 1, 1, 1]
141 145 [0, 1, 1, 1]
145 149 [1, 0, 0, 0]
149 153 [0, 1, 1, 1]
153 157 [1, 0, 0, 0]
157 161 [0, 1, 1, 1]
161 165 [1, 0, 0, 0]
165 169 [0, 1, 1, 1]
169 173 [0, 1, 1, 1]
173 177 [1, 0, 0, 0]
177 181 [0, 1, 1, 1]
181 185 [0, 1, 1, 1]
185 189 [0, 1, 1, 1]
189 193 [0, 1, 1, 1]
193 197 [0, 1, 1, 1]
197 201 [0, 1, 1, 1]
2

In [ ]:
df.to_excel("../data_output_table/auto_assess_distractors/EGEOutputWithPredictedLabels.xlsx")